<a href="https://colab.research.google.com/github/madanmohan-venkatesan/notebooks/blob/main/2011_03567.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [235]:
import numpy as np
from scipy.special import gamma

# 2. A sequential multinomial test


In [236]:
#CONSTANTS
CLASSES=2
VECTOR_SIZE=100

In [237]:
theta_null=np.array([1/CLASSES]*CLASSES) #Simplex
rng = np.random.default_rng()
vectors_null=rng.multinomial(1,theta_null,size=VECTOR_SIZE)

In [238]:
K=100
alpha_dirichlet=K*theta_null
rng = np.random.default_rng()
theta_alt=rng.dirichlet(alpha_dirichlet)
vectors_alt=rng.multinomial(1,theta_alt,size=VECTOR_SIZE)

In [239]:
s_n_null=np.sum(vectors_null,axis=0)
s_n_alt=np.sum(vectors_alt,axis=0)

In [240]:
#need some exceptions
def beta(v):
  numerator=np.prod(gamma(v))
  denominator=gamma(np.sum(v))
  return numerator/denominator

In [241]:
#need some exceptions
def bf_10(dirichlet_alpha,theta,counts):
  term1_numerator=beta(dirichlet_alpha+counts)
  term1_denominator=beta(dirichlet_alpha)
  term1=term1_numerator/term1_denominator

  term2=1/(theta**counts)

  return term1*term2

In [242]:
bf_10(alpha_dirichlet,theta_null,s_n_null)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-240-c183c70bffb2>:5: RuntimeWarning: invalid value encountered in scalar divide
  return numerator/denominator


array([nan, nan])

In [243]:
bf_10(alpha_dirichlet,theta_null,s_n_alt)

<ipython-input-240-c183c70bffb2>:5: RuntimeWarning: invalid value encountered in scalar divide
  return numerator/denominator


array([nan, nan])

For above bf_10, if theta_0 not equal theta then values will go to infinity (reject null hypothesis)
otherwise value is zero(accept null hypothesis)

Hyper paramters to play around with are K and vector size



In [244]:
theta_null_arr=[]
theta_alt_arr=[]
dirichlet_alpha_arr=[]
vetors_null_arr=[]
vectors_alt_arr=[]
for i in range(10):
  if i==0:
    K=10
  else:
    K=K#+(0.5*K)
  #generate null vectors
  theta_null=np.array([1/CLASSES]*CLASSES) #Simplex
  rng = np.random.default_rng()
  vectors_null=rng.multinomial(1,theta_null,size=VECTOR_SIZE)
  #generate alt vectors

  alpha_dirichlet=K*theta_null
  rng = np.random.default_rng()
  theta_alt=rng.dirichlet(alpha_dirichlet)
  vectors_alt=rng.multinomial(1,theta_alt,size=VECTOR_SIZE)

  theta_null_arr.append(theta_null)
  theta_alt_arr.append(theta_alt)
  dirichlet_alpha_arr.append(alpha_dirichlet)
  vetors_null_arr.append(vectors_null)
  vectors_alt_arr.append(vectors_alt)

In [245]:
#give some exceptions
on_theta_hist_arr=[]
for i in range(len(theta_null_arr)):
  on_theta_prev = None
  if i==0:
    on_theta_prev = np.array([1,1])
    on_theta_hist_arr.append(on_theta_prev)

  else:
    s_n_null=np.sum(vetors_null_arr[i],axis=0)
    s_n_alt=np.sum(vectors_alt_arr[i],axis=0)
    term1_numerator=beta(dirichlet_alpha_arr[i]+s_n_alt)
    term1_denominator=beta(dirichlet_alpha_arr[i])
    term1=term1_numerator/term1_denominator

    term2=1/(theta_null_arr[i]**s_n_alt)

    on_theta_hist_arr.append(on_theta_hist_arr[i-1]*term1*term2)

In [246]:
on_theta_hist_arr=np.array(on_theta_hist_arr)
on_theta_hist_arr

array([[1.00000000e+000, 1.00000000e+000],
       [3.86023753e-017, 2.47055202e-015],
       [2.05856398e-032, 3.29370237e-031],
       [1.88830525e-050, 5.06888080e-042],
       [1.73212819e-068, 7.80081186e-053],
       [1.21997952e-084, 8.79087892e-068],
       [6.85503887e-094, 4.60033871e-086],
       [1.49290209e-106, 1.52873174e-103],
       [8.36040257e-124, 2.46755531e-103],
       [1.82074278e-136, 8.19989648e-121]])

In [247]:
np.where(on_theta_hist_arr<(1/0.05),on_theta_hist_arr,np.inf)

array([[1.00000000e+000, 1.00000000e+000],
       [3.86023753e-017, 2.47055202e-015],
       [2.05856398e-032, 3.29370237e-031],
       [1.88830525e-050, 5.06888080e-042],
       [1.73212819e-068, 7.80081186e-053],
       [1.21997952e-084, 8.79087892e-068],
       [6.85503887e-094, 4.60033871e-086],
       [1.49290209e-106, 1.52873174e-103],
       [8.36040257e-124, 2.46755531e-103],
       [1.82074278e-136, 8.19989648e-121]])

# Optimization problem for theta

In [248]:
on_theta_hist_arr_=on_theta_hist_arr[on_theta_hist_arr<(1/0.05)].reshape(-1,2)

In [249]:
c = np.log(beta(alpha_dirichlet+s_n_alt)) - np.log(beta(alpha_dirichlet))
u=0.05

In [258]:
def constraint_1(theta,):
  return c+np.log(u)-np.sum(np.sum(vectors_alt_arr[-1],axis=0)*np.log(theta))
def constraint_2(theta):
  return np.sum(theta)-1
def objective(theta):
  return -theta

In [259]:
constraints=({'type':'ineq','fun':constraint_1},
             {'type':'eq','fun':constraint_2})

In [269]:
from scipy.optimize import minimize

def objective(theta, i):
    """
    Objective function to maximize theta_i.

    Parameters:
    - theta (array): Probability vector (simplex).
    - i (int): Index of the element to maximize.

    Returns:
    - float: Negative of theta[i] to convert minimization to maximization.
    """
    return -theta[i]

def constraint_sum(theta):
    """
    Constraint ensuring that the sum of theta is 1.

    Parameters:
    - theta (array): Probability vector (simplex).

    Returns:
    - float: Sum of theta - 1 (should be 0).
    """
    return np.sum(theta) - 1

def constraint_log_likelihood(theta, Sn, c):
    """
    Log-likelihood constraint.

    Parameters:
    - theta (array): Probability vector (simplex).
    - Sn (array): Observed count vector S_{n,i}.
    - c (float): Constant c + log u.

    Returns:
    - float: Left-hand side of the constraint.
    """
    return np.sum(Sn * np.log(theta)) - c

# Example data
Sn = np.sum(vectors_alt_arr[i],axis=0)  # Example counts for categories
c = np.log(beta(dirichlet_alpha_arr[-1]+np.sum(vectors_alt_arr[i],axis=0))) - np.log(beta(dirichlet_alpha_arr[-1]))  # Example constant for log Beta terms
u = 0.05  # Example confidence level
c += np.log(u)

# Initial guess for theta (uniform distribution)
initial_theta = np.full(len(Sn), 1 / len(Sn))

# Constraints
constraints = [
    {'type': 'eq', 'fun': constraint_sum},
    {'type': 'ineq', 'fun': constraint_log_likelihood, 'args': (Sn, c)}
]

# Index of theta to maximize
i = 0  # Change this to select a different element of theta

# Bounds to ensure theta remains within [0, 1]
bounds = [(0, 1) for _ in range(len(Sn))]

# Solve the optimization problem
result = minimize(objective, initial_theta, args=(i,), bounds=bounds, constraints=constraints, method='SLSQP')

# Display the result
if result.success:
    print("Optimal theta:", result.x)
    print(f"Maximized theta[{i}]:", result.x[i])
else:
    print("Optimization failed:", result.message)

Optimal theta: [0.45350482 0.54649518]
Maximized theta[0]: 0.45350482377083673


In [268]:
from scipy.optimize import minimize

def objective(theta, i):
    """
    Objective function to maximize theta_i.

    Parameters:
    - theta (array): Probability vector (simplex).
    - i (int): Index of the element to maximize.

    Returns:
    - float: Negative of theta[i] to convert minimization to maximization.
    """
    return theta[i]

def constraint_sum(theta):
    """
    Constraint ensuring that the sum of theta is 1.

    Parameters:
    - theta (array): Probability vector (simplex).

    Returns:
    - float: Sum of theta - 1 (should be 0).
    """
    return np.sum(theta) - 1

def constraint_log_likelihood(theta, Sn, c):
    """
    Log-likelihood constraint.

    Parameters:
    - theta (array): Probability vector (simplex).
    - Sn (array): Observed count vector S_{n,i}.
    - c (float): Constant c + log u.

    Returns:
    - float: Left-hand side of the constraint.
    """
    return np.sum(Sn * np.log(theta)) - c

# Example data
Sn = np.sum(vectors_alt_arr[i],axis=0)  # Example counts for categories
c = np.log(beta(dirichlet_alpha_arr[-1]+np.sum(vectors_alt_arr[i],axis=0))) - np.log(beta(dirichlet_alpha_arr[-1]))  # Example constant for log Beta terms
u = 0.05  # Example confidence level
c += np.log(u)

# Initial guess for theta (uniform distribution)
initial_theta = np.full(len(Sn), 1 / len(Sn))

# Constraints
constraints = [
    {'type': 'eq', 'fun': constraint_sum},
    {'type': 'ineq', 'fun': constraint_log_likelihood, 'args': (Sn, c)}
]

# Index of theta to maximize
i = 0  # Change this to select a different element of theta

# Bounds to ensure theta remains within [0, 1]
bounds = [(0, 1) for _ in range(len(Sn))]

# Solve the optimization problem
result = minimize(objective, initial_theta, args=(i,), bounds=bounds, constraints=constraints, method='SLSQP')

# Display the result
if result.success:
    print("Optimal theta:", result.x)
    print(f"Maximized theta[{i}]:", result.x[i])
else:
    print("Optimization failed:", result.message)

Optimal theta: [0.17201342 0.82798658]
Maximized theta[0]: 0.17201341724726557


In [270]:
theta_alt_arr[-1]

array([0.63899674, 0.36100326])